<a href="https://colab.research.google.com/github/tomonari-masada/course2024-stats1/blob/main/04_random_numbers_in_scipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 乱数の生成
* NumPyやscipyを使うと、多くの確率分布について、それに従う乱数を発生させることができる。
  * https://numpy.org/doc/stable/reference/random/generator.html
  * https://docs.scipy.org/doc/scipy/tutorial/stats/probability_distributions.html#random-number-generation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#発生させる乱数の個数
size = 10000

## 乱数の初期化

### NumPyのrandom number generator
* scipyの乱数生成はNumPyのそれに依存している。
* よって、NumPyでのrandom number generator作成方法をそのまま使う。
  * https://numpy.org/doc/stable/reference/random/generator.html#numpy.random.Generator
* 上のリンク先の説明によると
> Using just a small set of seeds to instantiate larger state spaces means that there are some initial states that are impossible to reach. This creates some biases if everyone uses such values.

In [ ]:
seed = np.random.SeedSequence().entropy
print(seed)
rng = np.random.default_rng(seed=seed)
rng

### random number generatorの使い方
* NumPyの場合は、以下のようにしてrandom number generatorを使う。

* 例１: 一様分布

In [ ]:
arr1 = rng.random((3, 3))
arr1

* 例2: ランダムな置換

In [ ]:
x = rng.permuted(np.arange(10))
x

## ベルヌーイ分布

* 一方のアイテムの出現確率が0.3のベルヌーイ分布を考える。
* このベルヌーイ分布に従う乱数を発生させる。
  * `rvs`というメソッドを使う。

In [ ]:
from scipy.stats import bernoulli

p = 0.3
r = bernoulli.rvs(p, size=size, random_state=rng)
pd.DataFrame({"outcomes":r}).value_counts().plot.bar();

* ベルヌーイ分布に従う乱数を自前で生成する方法
  * 一様乱数を利用する。

In [ ]:
def bernoulli_trial(p, rng):
  if rng.random() < p:
    return 1
  else:
    return 0

In [ ]:
outcomes = list()
for _ in range(size):
  outcomes.append(bernoulli_trial(0.3, rng))
outcomes = np.array(outcomes)
pd.DataFrame({"outcomes":outcomes}).value_counts().plot.bar();

* forループを使わずに書く。

In [ ]:
r = (rng.random(size=size) < 0.3) * 1
pd.DataFrame({"outcomes":r}).value_counts().plot.bar();

* この例のように・・・
  * 簡単な確率分布に従う乱数を使うと・・・
  * 別の確率分布に従う乱数を作ることができる場合がある。

## 二項分布

* 二項分布に従う「乱数」を生成するには・・・
  * ベルヌーイ試行を$n$回繰り返すということを、何回も繰り返せばよい。
  * つまり、「乱数」というよりも、「長さ$n$のランダムなアイテム列」をたくさん生成することになる。

* 確率質量関数を描いてみる。

In [ ]:
from scipy.stats import binom

n, p = 8, 0.4
x = np.arange(n+1)
plt.plot(x, binom.pmf(x, n, p), 'bo')
plt.vlines(x, 0, binom.pmf(x, n, p), colors='b');

* この二項分布に従うランダムな長さnのアイテム列を発生させ、頻度分布を描いてみる。
  * アイテムの出現順が違うだけのアイテム列は、全て同一視される。

In [ ]:
r = rng.binomial(n, p, size=size)
#r = binom.rvs(n, p, size=size, random_state=rng)
pd.DataFrame({"outcomes":r}).value_counts().sort_index().plot.bar();

## 単変量正規分布

### 標準正規分布
* 標準正規分布の確率密度関数を描いてみる。

In [ ]:
from scipy.stats import norm

x = np.linspace(-3, 3, 601)
plt.plot(x, norm.pdf(x), 'r-');

* 正規乱数を発生させて、相対頻度の分布を描いてみる。
* `density=True`とする。
  * こうすると、ヒストグラムの下の面積が1になるように描いてくれる。
  * 詳しくは https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html

In [ ]:
r = rng.standard_normal(size=size)
#r = norm.rvs(size=size, random_state=rng)
plt.plot(x, norm.pdf(x), 'r-') # 上と同じ
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);

* 問: 下の2つの値は、正規分布の、どういう値でしょうか？
  * ヒント: ppf = percent point function

In [ ]:
print(norm.ppf(0.01))
print(norm.ppf(0.99))

### 一般の正規分布
  * 平均パラメータを0でない適当な値にする。
  * 標準偏差パラメータも適当な値にする。

* 確率密度関数を描いてみる。

In [ ]:
# これは標準正規分布
plt.plot(x, norm.pdf(x), 'r-')

# こちらが標準正規分布でない正規分布
loc, scale = 0.2, 0.5
plt.plot(x, norm.pdf(x, loc=loc, scale=scale), 'b-');

* 標準正規分布でない正規分布に従う乱数を発生させる。

In [ ]:
r = rng.normal(size=size, loc=loc, scale=scale)
#r = norm.rvs(size=size, loc=loc, scale=scale)
plt.plot(x, norm.pdf(x, loc=loc, scale=scale), 'b-')
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);

* 正規乱数から、任意の正規分布に従う乱数を、作ることができる。
  * 標準偏差を掛けて、平均を足せばよい。

In [ ]:
r = norm.rvs(size=size) * scale + loc
#r = rng.standard_normal(size=size) * scale + loc
plt.plot(x, norm.pdf(x, loc=loc, scale=scale), 'b-')
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);

## 二変量正規分布

* 二変量なので、密度関数の高さを、平面上に等高線で可視化することにする。

* 等高線図を描く準備
  * xy平面にグリッドを設定する。

In [ ]:
x = np.linspace(-3, 3, 601)
y = np.linspace(-3, 3, 601)
X, Y = np.meshgrid(x, y)
pos = np.stack([X, Y], axis=2)

* 二変量正規分布の等高線図

In [ ]:
from scipy.stats import multivariate_normal

cov = [[1, 0.2], [0.2, 1]]
ax = plt.subplot(1,1,1)
ax.contourf(X, Y, multivariate_normal.pdf(pos, cov=cov))
ax.set_aspect('equal');

* 二変量正規分布に従う「乱数」の散布図を描く。
  * 二変量なので、「乱数」と言うよりも、「ランダムな二次元ベクトル」を発生させることになる。

In [ ]:
r = multivariate_normal.rvs(size=size, cov=cov, random_state=rng)
ax = plt.subplot(1,1,1)
ax.plot(r[:,0], r[:,1], 'o', alpha=0.1)
ax.axis('equal');

# 本日の課題
* 適当に、二変量正規分布を設定する。
  * つまり、平均ベクトルと、共分散行列を設定する。
* まず、その二変量正規分布の確率密度関数の等高線を描く。
  * 上の例を参考にしてください。
* 次に、その二変量正規分布に従う乱数を10000個発生させる。
* そして、サンプルの散布図を描画する。
  * これも、上の例を参考にしてください。
* ただし、`rng.standard_normal()`だけを使うこと。
  * NumPyやscipyの`multivariate_normal`は使わないこと。

## 課題のヒント
* ヒント1: まず、正規乱数をたくさん発生させましょう。
  * 何個の正規乱数が必要かは、考えましょう。
* ヒント2: コレスキー分解を使いましょう。
  * コレスキー分解は、以下のようにすると簡単に求まります。

In [ ]:
cov = np.array([[1, 0.2], [0.2, 1]])
cov_L = np.linalg.cholesky(cov)
print(cov_L)
print(cov_L @ cov_L.T)



---

---

# Appendix

## 逆関数法
* 指数分布の例

In [ ]:
def F_inverse(u, l):
  return - np.log(1 - u) / l

In [ ]:
from scipy.stats import expon

l = 2.0
x = np.linspace(0, 5, 501)
r = F_inverse(rng.random(size), l)
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);
plt.plot(x, expon.pdf(x, scale=1/l), 'b-');

## ベータ分布

* ベータ分布に従う乱数は・・・
  * 二つのパラメータが1より大きい場合はガンマ分布に従う乱数から作ることができる。
  * そうでない場合は、特殊なアルゴリズムが必要。
  * 下記の`double random_beta()`という関数を参照。
    * https://github.com/numpy/numpy/blob/main/numpy/random/src/distributions/distributions.c

### ガンマ分布
* shapeパラメータが1より大きい場合を考える。

In [ ]:
from scipy.stats import gamma

a = 3.0
x = np.linspace(0, 10, 501)
plt.plot(x, gamma.pdf(x, a), 'r-');

In [ ]:
r = rng.gamma(a, size=size)
#r = gamma.rvs(a, size=size, random_state=rng)
plt.plot(x, gamma.pdf(x, a), 'r-') # 上と同じ
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);

### ベータ分布

* 確率密度関数を描く。

In [ ]:
from scipy.stats import beta

a = 3.0
b = 2.0
x = np.linspace(0, 1, 101)
plt.plot(x, beta.pdf(x, a, b), 'r-');

In [ ]:
r = rng.beta(a, b, size=size)
#r = beta.rvs(a, b, size=size, random_state=rng)
plt.plot(x, beta.pdf(x, a, b), 'r-') # 上と同じ
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);

* shapeパラメータがaとbの二つのガンマ分布に従う乱数を発生させる。
* それらを規格化すると、ベータ分布に従う乱数が得られる。

In [ ]:
ra = rng.gamma(a, size=size)
rb = rng.gamma(b, size=size)
r = ra / (ra + rb)
plt.plot(x, beta.pdf(x, a, b), 'r-') # 上と同じ
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);

## カテゴリカル分布

* 確率質量関数

In [ ]:
p = np.array([0.3, 0.25, 0.2, 0.15, 0.1])
k = len(p)

plt.plot(np.arange(k), p, 'ro')
plt.vlines(np.arange(k), 0, p, colors='r');

* NumPyを使う。

In [ ]:
r = rng.choice(np.arange(k), p=p, size=size)
pd.DataFrame({"outcomes":r}).value_counts().plot.bar(color='r');

* 一様乱数を使う。

In [ ]:
r = (p.cumsum().reshape(1, -1) < rng.random(size=size).reshape(-1, 1)).sum(-1)
pd.DataFrame({"outcomes":r}).value_counts().plot.bar(color='r');

### Gumbel Max Trick

In [ ]:
from scipy.stats import gumbel_r

x = np.linspace(-2, 8, 1001)
plt.plot(x, gumbel_r.pdf(x), 'r-');

In [ ]:
r = rng.gumbel(size=size)
#r = gumbel_r.rvs(size=size, random_state=rng)
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2)
plt.plot(x, gumbel_r.pdf(x), 'r-') # 上と同じ
plt.hist(r, density=True, bins='auto', histtype='stepfilled', alpha=0.2);

In [ ]:
r = (np.log(p + 1e-10) + rng.gumbel(size=(size, k))).argmax(-1)
pd.DataFrame({"outcomes":r}).value_counts().plot.bar(color='r');